In [ ]:
import pandas as pd
import numpy as np

In [ ]:
match = pd.read_csv('matches.csv')
delivery = pd.read_csv('deliveries.csv')

In [ ]:
total_score_df = delivery.groupby(['match_id', 'inning']).sum()['total_runs'].reset_index()


In [ ]:
total_score_df = total_score_df[total_score_df['inning'] == 1]


In [ ]:
total_score_df

In [ ]:
match_df = match.merge(total_score_df[['match_id', 'total_runs']], left_on='id', right_on='match_id')

In [ ]:
match_df

In [ ]:
match_df['team1'].unique()


In [ ]:
teams = ['Sunrisers Hyderabad', 'Mumbai Indians',
         'Royal Challengers Bangalore',
         'Kolkata Knight Riders', 'Kings XI Punjab',
         'Chennai Super Kings', 'Rajasthan Royals',
         'Delhi Capitals']


In [ ]:
match_df['team1']=match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2']=match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1']=match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2']=match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [ ]:
match_df=match_df[match_df['team1'].isin(teams)]
match_df=match_df[match_df['team2'].isin(teams)]

In [ ]:
match_df['dl_applied'].value_counts()

In [ ]:
match_df=match_df[match_df['dl_applied']==0]

In [ ]:
match_df

In [ ]:
match_df=match_df[['match_id','city','winner','total_runs']]

In [ ]:
delivery_df=match_df.merge(delivery,on='match_id')

In [ ]:
delivery_df=delivery_df[delivery_df['inning']==2]

In [ ]:
delivery_df

In [ ]:
delivery_df['current_score']=delivery_df.groupby('match_id').cumsum()['total_runs_y']

In [ ]:
delivery_df['runs_left']=delivery_df['total_runs_x']-delivery_df['current_score']

In [ ]:
delivery_df['balls_left']=126- (delivery_df['over']*6 + delivery_df['ball'])

In [ ]:
delivery_df
# delivery_df.drop(columns=['wickets'],inplace=True)

In [ ]:
delivery_df['player_dismissed']=delivery_df['player_dismissed'].fillna("0")
delivery_df['player_dismissed']=delivery_df['player_dismissed'].apply(lambda x:x if x =="0" else "1")
delivery_df['player_dismissed']=delivery_df['player_dismissed'].astype(int)
wickets=delivery_df.groupby('match_id').cumsum()['player_dismissed'].values
delivery_df['wickets']=10-wickets 
delivery_df.head()

In [ ]:
delivery_df.head()

In [ ]:
# calculate Current Rate
delivery_df['crr']=(delivery_df['current_score']*6)/(120-delivery_df['balls_left'])

In [ ]:
# calculate required Run Rate
delivery_df['rrr']=(delivery_df['runs_left']*6)/(delivery_df['balls_left'])

In [ ]:
def result(row):
    if row['batting_team']== row['winner']:
        return 1
    else:
        return 0

In [ ]:
delivery_df

In [ ]:
final_df=delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets_left','crr','rrr','result']]

In [ ]:
final_df

In [ ]:
final_df.to_csv('finaldf.csv',index='False')

In [212]:
final_df.sample(final_df.shape[0])

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,crr,rrr,result
111074,Mumbai Indians,Royal Challengers Bangalore,Mumbai,206,96,9,7.250000,12.875000,0
30098,Kings XI Punjab,Delhi Daredevils,Bloemfontein,58,51,7,5.391304,6.823529,1
22260,Mumbai Indians,Deccan Chargers,Durban,159,104,9,3.375000,9.173077,0
79459,Rajasthan Royals,Royal Challengers Bangalore,Jaipur,139,96,9,8.000000,8.687500,1
14689,Deccan Chargers,Kolkata Knight Riders,Hyderabad,187,101,9,5.368421,11.108911,0
...,...,...,...,...,...,...,...,...,...
2308,Kolkata Knight Riders,Delhi Daredevils,Delhi,149,109,9,10.363636,8.201835,1
51046,Rajasthan Royals,Kings XI Punjab,Chandigarh,150,84,8,7.500000,10.714286,0
21088,Royal Challengers Bangalore,Chennai Super Kings,Port Elizabeth,113,55,5,6.092308,12.327273,0
60813,Chennai Super Kings,Royal Challengers Bangalore,Chennai,194,105,10,4.400000,11.085714,1


In [ ]:
dsnjdus